In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision as vision
from torchvision.datasets import MNIST
from torchvision.utils import make_grid

In [3]:
data=MNIST(root="data",download=True,transform=vision.transforms.ToTensor())
data

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


/opt/conda/conda-bld/pytorch_1587428190859/work/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

# Model

In [4]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [5]:
class feedf(nn.Module):
    def __init__(self,ip_size,h_size,op_size):
        super().__init__()
        self.linear1=nn.Linear(ip_size,h_size)
        self.linear2=nn.Linear(h_size,op_size)
    
    def forward(self,xb):
        xb=xb.view(xb.size(0),-1)
        out=self.linear1(xb)
        out=nn.functional.relu(out)
        out=self.linear2(out)
        return out
    
    def training_step(self,batch):
        img,l=batch
        pred=self(img)
        loss=nn.functional.cross_entropy(pred,l)
        return loss
    
    def validation_step(self,batch):
        img,l=batch
        pred=self(img)
        loss=nn.functional.cross_entropy(pred,l)
        acc=accuracy(pred,l)
        return {"val_loss":loss,"val_acc":acc}
    
    def validation_epoch_end(self,output):
        loss=[x["val_loss"] for x in output]      
        acc=[x["val_acc"] for x in output]      
        loss=torch.stack(loss).mean()
        acc=torch.stack(acc).mean()
        return {"loss":loss.item(),"acc":acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))
    
            
            
        

In [6]:
ts,vs=torch.utils.data.random_split(data,[50000,10000])
tl=DataLoader(ts,batch_size=128,pin_memory=True,num_workers=2)

In [7]:
model=feedf(784,32,10)

In [8]:
for i,j in tl:
    pred=model(i)
    loss=nn.functional.cross_entropy(pred,j)
    print(loss)
    break
    

AttributeError: module 'torch.nn.functional' has no attribute 'relue'